# All steps of the receipts extraction pipeline
written by: Jiaen LIU 

This notebook will walk through all the steps of the pipeline to extarct the date from the receipts
and automatically select the best hyper-parameters for the preprocessing by getting the best accuracy
of the date extraction.  
The following steps are included:  
INPUT FOLDER  
STEP 1. MobileNetV3-Large  
STEP 2. Preprocess  
STEP 3. OCR  
STEP 4. Regex + Compare to ground truth  
1. is it part of the candidate?  
2. is the best candidate the ground truth?  
3. if we take n best candidates, which n is the best? plot the accuracy vs n  
STEP 5. Aggregate results use streamlit to visualize the results (thumnails of input and preprocessed, OCR results, regex results, final results(whether the ground in the candidates), accuracy vs n)
OUTPUT FOLDER  
STEP 6. Hyper-parameters selection  
Let's start by importing the necessary libraries  

In [88]:
import os
import gc
import io
import cv2
import base64
import pathlib
import numpy as np
from PIL import Image
from glob import glob
import re
from dateutil import parser

import pytesseract
from skimage.filters import threshold_local

import torch
import torchvision.transforms as torchvision_T
from torchvision.models.segmentation import deeplabv3_mobilenet_v3_large

In [13]:
def load_model(model_path=None,num_classes=2, model_name="mbv3", device=torch.device("cpu")):
    """Load the model from the given path.
    Args:
        model_path (str): Path to the model.
        num_classes (int): Number of classes.
        model_name (str): Name of the model.
        device (torch.device): Device to load the model.
    Returns:
        model (torch.nn.Module): Loaded model.
    """
    if model_name == "mbv3":
        model = deeplabv3_mobilenet_v3_large(num_classes=num_classes, aux_loss=True)
    else:
        raise ValueError(f"Model {model_name} not supported.")
    if model_path is not None:
        model.to(device)
        checkpoints = torch.load(model_path, map_location=device)
        model.load_state_dict(checkpoints, strict=False)
        model.eval()
        _ = model(torch.rand(1, 3, 384, 384))
    else:
        raise ValueError("Model path not provided.")

    return model


In [14]:
# Test load_model
model = load_model(model_path="model_mbv3_iou_mix_2C_aux_e3_pretrain.pth", num_classes=2, model_name="mbv3", device=torch.device("cpu"))

In [15]:
def image_preprocess_transforms(mean=(0.4611, 0.4359, 0.3905), std=(0.2193, 0.2150, 0.2109)):
    # Preprocessing transforms. Convert to tensor and normalize.
    common_transforms = torchvision_T.Compose(
        [
            torchvision_T.ToTensor(),
            torchvision_T.Normalize(mean, std),
        ]
    )
    return common_transforms

In [16]:
def image_resize(image, width = None, height = None, inter = cv2.INTER_AREA):
    """Resize the image to the given width and height.
    Args:
        image (np.ndarray): Image to resize.
        width (int): Width to resize to.
        height (int): Height to resize to.
        inter (int): Interpolation method.
    Returns:
        image (np.ndarray): Resized image.
    """
    # initialize the dimensions of the image to be resized and
    # grab the image size
    dim = None
    (h, w) = image.shape[:2]

    # if both the width and height are None, then return the
    # original image
    if width is None and height is None:
        return image

    # check to see if the width is None
    if width is None:
        # calculate the ratio of the height and construct the
        # dimensions
        r = height / float(h)
        dim = (int(w * r), height)

    # otherwise, the height is None
    else:
        # calculate the ratio of the width and construct the
        # dimensions
        r = width / float(w)
        dim = (width, int(h * r))

    # resize the image
    resized = cv2.resize(image, dim, interpolation = inter)

    # return the resized image
    return resized

In [17]:
# Add a scanner effect to the image.
def add_scanner_effect(img, parameters={
    "blur": (5,5),
    "blur_iterations": 1,
    "erode_kernel": np.ones((3,3), np.uint8),
    "erode_iterations": 1,
    "dilate_kernel": np.ones((3,3), np.uint8),
    "dilate_iterations": 1,
    "erode_kernel_2": np.ones((3,3), np.uint8),
    "erode_iterations_2": 1,
    "dilate_kernel_2": np.ones((3,3), np.uint8),
    "dilate_iterations_2": 1,
    "threshold_local_block_size": 31,
    "threshold_local_offset": 7,
    # "threshold_local_mode": 'reflect',
    "sharpen_kernel": np.array([[0,-1,0], [-1,7,-1], [0,-1,0]]),
}):
    # Convert the image to grayscale.
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Apply a Gaussian blur to the image.
    img = cv2.GaussianBlur(img, parameters["blur"], parameters["blur_iterations"])

    img = cv2.erode(img, parameters["erode_kernel"], iterations=parameters["erode_iterations"])
    img = cv2.dilate(img, parameters["dilate_kernel"], iterations=parameters["dilate_iterations"])

    # Apply a threshold to the image.
    T = threshold_local(img, parameters["threshold_local_block_size"], offset=parameters["threshold_local_offset"], method="gaussian")
    img = (img > T).astype("uint8") * 255
    
    # img = cv2.erode(img, parameters["erode_kernel_2"], iterations=parameters["erode_iterations_2"])
    # img = cv2.dilate(img, parameters["dilate_kernel_2"], iterations=parameters["dilate_iterations_2"])

    # Sharpen
    img = cv2.filter2D(src=img, ddepth=-1, kernel=parameters["sharpen_kernel"])
    return img

In [18]:
def image_preprocess_transforms(mean=(0.4611, 0.4359, 0.3905), std=(0.2193, 0.2150, 0.2109)):
    # Preprocessing transforms. Convert to tensor and normalize.
    common_transforms = torchvision_T.Compose(
        [
            torchvision_T.ToTensor(),
            torchvision_T.Normalize(mean, std),
        ]
    )
    return common_transforms

In [19]:
def order_points(pts):
    """Rearrange coordinates to order:
    top-left, top-right, bottom-right, bottom-left"""
    rect = np.zeros((4, 2), dtype="float32")
    pts = np.array(pts)
    s = pts.sum(axis=1)
    # Top-left point will have the smallest sum.
    rect[0] = pts[np.argmin(s)]
    # Bottom-right point will have the largest sum.
    rect[2] = pts[np.argmax(s)]

    diff = np.diff(pts, axis=1)
    # Top-right point will have the smallest difference.
    rect[1] = pts[np.argmin(diff)]
    # Bottom-left will have the largest difference.
    rect[3] = pts[np.argmax(diff)]
    # return the ordered coordinates
    return rect.astype("int").tolist()

In [20]:
def find_dest(pts):
    (tl, tr, br, bl) = pts
    # Finding the maximum width.
    widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
    maxWidth = max(int(widthA), int(widthB))

    # Finding the maximum height.
    heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
    maxHeight = max(int(heightA), int(heightB))
    # Final destination co-ordinates.
    destination_corners = [[0, 0], [maxWidth, 0], [maxWidth, maxHeight], [0, maxHeight]]

    return order_points(destination_corners)

In [21]:
def scan(image_true=None, trained_model=None, image_size=384, BUFFER=10):
    """Scan the image and return the scanned image
    Args:
        image_true (np.array): Image to be scanned
        trained_model (torch.nn.Module): Trained model
        image_size (int): Size of the image to be fed to the model
        BUFFER (int): Buffer to be added to the image
    Returns:
        scanned_image (np.array): Scanned image
    """
    global preprocess_transforms

    IMAGE_SIZE = image_size
    half = IMAGE_SIZE // 2

    imH, imW, C = image_true.shape

    # Resizing the image to the size of input to the model. (384, 384)
    image_model = cv2.resize(image_true, (IMAGE_SIZE, IMAGE_SIZE), interpolation=cv2.INTER_NEAREST)

    scale_x = imW / IMAGE_SIZE
    scale_y = imH / IMAGE_SIZE

    # Converting the image to tensor and normalizing it.
    image_model = preprocess_transforms(image_model)
    image_model = torch.unsqueeze(image_model, dim=0)

    with torch.no_grad():
        # Out: the output of the model
        out = trained_model(image_model)["out"].cpu()

    del image_model
    gc.collect()

    out = torch.argmax(out, dim=1, keepdims=True).permute(0, 2, 3, 1)[0].numpy().squeeze().astype(np.int32)
    r_H, r_W = out.shape

    _out_extended = np.zeros((IMAGE_SIZE + r_H, IMAGE_SIZE + r_W), dtype=out.dtype)
    _out_extended[half : half + IMAGE_SIZE, half : half + IMAGE_SIZE] = out * 255
    out = _out_extended.copy()

    del _out_extended
    # Garbage collection
    gc.collect()

    # Edge Detection.
    canny = cv2.Canny(out.astype(np.uint8), 225, 255)
    canny = cv2.dilate(canny, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5)))
    contours, _ = cv2.findContours(canny, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
    # Finding the largest contour. (Assuming that the largest contour is the document)
    page = sorted(contours, key=cv2.contourArea, reverse=True)[0]

    # ==========================================
    epsilon = 0.02 * cv2.arcLength(page, True)
    corners = cv2.approxPolyDP(page, epsilon, True)

    corners = np.concatenate(corners).astype(np.float32)

    corners[:, 0] -= half
    corners[:, 1] -= half

    corners[:, 0] *= scale_x
    corners[:, 1] *= scale_y

    # check if corners are inside.
    # if not find smallest enclosing box, expand_image then extract document
    # else extract document

    if not (np.all(corners.min(axis=0) >= (0, 0)) and np.all(corners.max(axis=0) <= (imW, imH))):

        left_pad, top_pad, right_pad, bottom_pad = 0, 0, 0, 0

        rect = cv2.minAreaRect(corners.reshape((-1, 1, 2)))
        box = cv2.boxPoints(rect)
        box_corners = np.int32(box)
        #     box_corners = minimum_bounding_rectangle(corners)

        box_x_min = np.min(box_corners[:, 0])
        box_x_max = np.max(box_corners[:, 0])
        box_y_min = np.min(box_corners[:, 1])
        box_y_max = np.max(box_corners[:, 1])

        # Find corner point which doesn't satify the image constraint
        # and record the amount of shift required to make the box
        # corner satisfy the constraint
        if box_x_min <= 0:
            left_pad = abs(box_x_min) + BUFFER

        if box_x_max >= imW:
            right_pad = (box_x_max - imW) + BUFFER

        if box_y_min <= 0:
            top_pad = abs(box_y_min) + BUFFER

        if box_y_max >= imH:
            bottom_pad = (box_y_max - imH) + BUFFER

        # new image with additional zeros pixels
        image_extended = np.zeros((top_pad + bottom_pad + imH, left_pad + right_pad + imW, C), dtype=image_true.dtype)

        # adjust original image within the new 'image_extended'
        image_extended[top_pad : top_pad + imH, left_pad : left_pad + imW, :] = image_true
        image_extended = image_extended.astype(np.float32)

        # shifting 'box_corners' the required amount
        box_corners[:, 0] += left_pad
        box_corners[:, 1] += top_pad

        corners = box_corners
        image_true = image_extended

    corners = sorted(corners.tolist())
    corners = order_points(corners)
    destination_corners = find_dest(corners)
    M = cv2.getPerspectiveTransform(np.float32(corners), np.float32(destination_corners))

    final = cv2.warpPerspective(image_true, M, (destination_corners[2][0], destination_corners[2][1]), flags=cv2.INTER_LANCZOS4)
    final = np.clip(final, a_min=0, a_max=255)
    final = final.astype(np.uint8)

    return final

In [22]:
# Extract the document from the image
def extract_document(image, model_path, image_size=384):
    """ Extract the document from the image
    Args:
        image (np.array): Image to be scanned
    Returns:
        scanned_image (np.array): Scanned image
    """
    # Load the model
    model = load_model(model_path)
    final = scan(image_true=image, trained_model=model, image_size=image_size)
    return final
    

In [31]:
# OCR the document by using the pytesseract library.
def ocr_document(image, lang="fra", width=600):
    """
    OCR the document by using the pytesseract library.
    Args:
        image (np.array): Image to be OCR'd
        lang (str): Language of the document
        width (int): Width of the image
    Returns:
        text (str): Text extracted from the document
        image (np.array): Image with the scanner effect applied
    """
   
    imW = image.shape[1] # Get the size of the image
    if imW > width:
        # Resize the image to the width of 600 if the width is greater than 600 do that.
        image = image_resize(image, width=width)
    # Add a scanner effect to the image
    image = add_scanner_effect(image)

    return pytesseract.image_to_string(image, lang=lang),image

In [86]:
# Using the regex to get the date from the text.
# TODO: find a way to get the true date from current candidates.
def get_date(text):
    """
    Using the regex to get the date from the text.
    Args:
        text (str): Text to be searched
    Returns:
        date (str): Date found in the text
    """
    # Lower the text
    text = text.lower()
    # Remove all the new line characters
    text = text.replace("\n", " ")
    # print(text)
    # Get the date from the text
    date = [x.group() for x in re.finditer( r'((?P<day>\d{1,2}))(\D{0,5})((?:(?:jan(?:v(?:ier)?)?)\.?)|(?:(?:fév(?:r(?:ier)?)?)\.?)|(?:(?:mar(?:s)?)\.?)|(?:(?:avr(?:i(?:l)?)?)\.?)|(?:mai\.?)|(?:(?:jui(?:n)?)\.?)|(?:(?:jui(?:l(?:let)?)?)\.?)|(?:(?:aoû(?:t)?)\.?)|(?:(?:sep(?:t(?:embre)?)?)\.?)|(?:(?:oct(?:o(?:bre)?)?)\.?)|(?:(?:nov(?:e(?:mbre)?)?)\.?)|(?:(?:déc(?:e(?:mbre)?)?)\.?)|(?P<month>\d{1,2}))(\D{0,5})((?P<year>\d{2,4}))', text)]
    return date

In [85]:
[x.group() for x in re.finditer( r'((?P<day>\d{1,2}))(\D{0,5})((?:(?:jan(?:v(?:ier)?)?)\.?)|(?:(?:fév(?:r(?:ier)?)?)\.?)|(?:(?:mar(?:s)?)\.?)|(?:(?:avr(?:i(?:l)?)?)\.?)|(?:mai\.?)|(?:(?:jui(?:n)?)\.?)|(?:(?:jui(?:l(?:let)?)?)\.?)|(?:(?:aoû(?:t)?)\.?)|(?:(?:sep(?:t(?:embre)?)?)\.?)|(?:(?:oct(?:o(?:bre)?)?)\.?)|(?:(?:nov(?:e(?:mbre)?)?)\.?)|(?:(?:déc(?:e(?:mbre)?)?)\.?)|(?P<month>\d{1,2}))(\D{0,5})((?P<year>\d{2,4}))', '''— TANG FRERES GOURMET 48 AVENUE D'IVRY 75013 PARIS Tel : 01 45 70 80 00 siret : 306 243 833 00044 TVA INTRA : FR8S 306 243 833 *$‘t>t*#'**ﬂ**********ñ***1A‘A4w***—k********** | samedi 05/11/2022 17:32:05 Ticket 371 Caisse No 6 ; Caissiere Ha ; dan Magasin G03 CODE LIBELLE PRIX NET EURO QUANTITE 0.544 x 21.90 336469206110 ECHINE DE PORC LAQUE 11.91 — ia Total TTC Euros 11.91 Nombre d'articles : 0.544 Détail Tva : _TVA à 5.50% 0.62 Reglement(s) $ Paiement Espèces EURO 15.00 À Rendu Espèces EURg - 3.09 NeoLog - Logicie] NeoPos Version 1.0.104 Signature numérique : B-NEOP-EyxA **************************************%*** Horaires du Magasin : Fermé le Lundi Mardi au Vendredi : Sh00 - 20h00 Samedi : 8h00 - 20h00 Dimanche : 8h00 - 13h00 **************************""*****’Ï‘***Î‘***** Merci de Votre Visite et à Bientôt \ ('3\‘,—' F ù''')]

['75013',
 '01 45 70',
 '80 00',
 '306 243',
 '833 0004',
 '8S 306',
 '243 833',
 '05/11/2022',
 '17:32:05',
 '0.544',
 '21.90 3364',
 '69206110',
 '11.91',
 '11.91',
 '0.544',
 '50% 0.62',
 '15.00',
 '1.0.104',
 '00 - 20h00',
 '8h00 - 20',
 '8h00 - 13']

In [25]:
# Some hyperparameters
IMAGE_SIZE = 384
preprocess_transforms = image_preprocess_transforms()
image = None
final = None
result = None
INPUT_FOLDER = "input"
OUTPUT_FOLDER = "output"


In [26]:
# Load all pictures in the folder
receipt_files = glob(INPUT_FOLDER + "/*")

In [75]:
result

'à s\n* (3\n«& city\n. Ld\nCRF—CITY LA ROCHELLE\n33 RUE DE LÀ SCIERIE\n\n17000 LÀ ROŒCHELLE\nTel : 05.46.27.02.12\n\nDESCR/PTISK [03 KOWTÉ\n\n+1509 {DV TR.PAL PL 2.3\n=;804 CKIPS CNDULEE 1.4\n+1908 {DW TR. JBX C 3.7\n\n3 fL.IICLECS) TOTAL A PAYER 6.99\nCB EMV SANS CONTACT EUR 6.99€\n\n01 05 093 000009— 25/02/2017 .— rp:Cs:s\n\n(S MERCI DE VOTRE VYISITE\nÀ BIENTOT\n\n'

: 

In [98]:
test = get_date(result)
test.append("1701-12")
# test.append("")

In [99]:
import pandas as pd
out = pd.to_datetime(test, errors='coerce').dropna().strftime('%d-%b-%Y %H:%M').tolist()
out

/var/folders/k5/j33smc310f1038b26bybvjvm0000gn/T/ipykernel_38011/1606942962.py:2: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  out = pd.to_datetime(test, errors='coerce').dropna().strftime('%d-%b-%Y %H:%M').tolist()


['01-May-1993 00:00', '25-Feb-2017 00:00', '01-Dec-1701 00:00']

In [91]:
# step1: if the ground truth is in the candidate list, then we say it is good
# step2: is the best candidate is in the ground truth, then we say it is good
# step3: if the ground truth is in the best candidates, then we say it is good

'25/02/2017'

In [100]:
for time in test:
    try:
        date = parser.parse(time)
    except:
        pass
    print(date)

2017-02-25 00:00:00
2017-02-25 00:00:00
2022-11-02 00:00:00
1509-11-30 00:00:00
1509-11-30 00:00:00
1509-11-30 00:00:00
1509-11-30 00:00:00
1993-05-30 00:00:00
1993-05-30 00:00:00
2017-02-25 00:00:00
1701-12-30 00:00:00


In [32]:
# For each picture in the folder do the following steps
for receipt_file in receipt_files:
    # Read the image
    image = cv2.imread(receipt_file)
    # Extract the document
    final = extract_document(image, "model_mbv3_iou_mix_2C_aux_e3_pretrain.pth", IMAGE_SIZE)
    # OCR the document
    result,scanned_image = ocr_document(final)
    # Save the image

    # create the output folder if it doesn't exist
    if not os.path.exists(OUTPUT_FOLDER):
        os.makedirs(OUTPUT_FOLDER)

    cv2.imwrite(OUTPUT_FOLDER + "/" + receipt_file.split("/")[-1], scanned_image)
    # Print the result
    print(result)
    # write the result in a text file
    with open(OUTPUT_FOLDER + "/" + receipt_file.split("/")[-1].split(".")[0] + ".txt", "w") as f:
        f.write(result)
    
    # Get the date from the text
    date = get_date(result)
    print(date)
    # write the date in a text file
    with open(OUTPUT_FOLDER + "/" + receipt_file.split("/")[-1].split(".")[0] + "_date.txt", "w") as f:
        f.write(str(date))


LESCAIPTICA dt voutet
3208 SLB.WÏTM [AL
PORNE aoLDEA 990
g 36004 X _1.50(M

comssnamnett* ____,.—,—-,;;.—-—______.—_...;—_—.::—‘--—-- st
2 ONTIELECS) TOTAL ® POYER 5 04t
-:_...-———--—-,:—"‘.,..___..-—__:—;: —-_:“.—-—-.:‘_:”-*“'::—-*“‘….———*“'
CARTE BANCAIRE ç  ER 5.04
j9:79:0

[('3', '3', '', '2', '2', '', '08', '08'), ('99', '99', '', '0', '0', '\ng ', '3600', '3600'), ('4', '4', ' X _', '1', '1', '.', '50', '50'), ('04', '04', '\nj', '9', '9', ':', '79', '79')]
Carref
ae<gg;(®

CRF—CITY LA ROCHELLE
33 RUE DE LA SCIERIE
17000 _ LA E
Tel : 05.46.27.02.12

TESCRIPTIOIL L NOTRAT
2008 SCHKOSONS K1 3126
75 CAD Gt RSE KOD 4156
PREFOU CHEURE 4806
r_:—=:::æ:‘;:ﬂlﬁx:æ:£ï—".:z::ﬁ::

3 MIILEGS) TOTAL A PATER 1206

CARTE BANCATRE EMY — RR 12.07€
9012 . 004/ c00119 —— M/N0/06 19:15:57

MERCT DE VOTRE VISITE
A BIENTOT

[('17', '17', '', '0', '0', '', '00', '00'), ('05', '05', '.', '46', '46', '.', '27', '27'), ('0', '0', '', '2', '2', '.', '12', '12'), ('2', '2', '', '0', '0', '', '08', '08'), ('1'

In [6]:
import cv2
import numpy as np

# Load the image
img = cv2.imread("output.png")

# Create a grayscale version of the image
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Define a kernel for the scanner effect
scanner_kernel = np.array([[-1, -1, -1], [-1, 9, -1], [-1, -1, -1]])

# Apply the scanner effect to the grayscale image
scanner = cv2.filter2D(gray, -1, scanner_kernel)

# resize the image to scanner size
# img = cv2.resize(img, (img.shape[1], img.shape[0]))


# Combine the original image with the scanner effect
result = cv2.addWeighted(gray, 0.5, scanner, 0.5, 0)

# Save the resulting image
cv2.imwrite("result.png", result)

True

In [3]:
import cv2
import numpy as np

# Load the image
image = cv2.imread('/Users/liujiaen/Documents/Text_Recognition/Document-Scanner-Custom-Semantic-Segmentation-using-PyTorch-DeepLabV3/test_images/IMG_3481.jpg')

# Convert the image to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Apply a Gaussian blur to the image to reduce noise

blur = cv2.GaussianBlur(gray, (5, 5), 0)

# Use Canny edge detection to find the edges in the image
edges = cv2.Canny(blur, 50, 150)

# Run the Hough transform on the edge-detected image to detect lines
lines = cv2.HoughLinesP(edges, 1, np.pi/180, 100, minLineLength=100, maxLineGap=10)

# create a white image with the same size as the original image
mask = np.zeros_like(image)
mask.fill(255)

# Draw the lines on the white image
for line in lines:
    x1, y1, x2, y2 = line[0]
    cv2.line(mask, (x1, y1), (x2, y2), (0, 0, 0), 2)

cv2.imwrite('mask.png', mask)

# Draw the lines on the original image
for line in lines:
    x1, y1, x2, y2 = line[0]
    cv2.line(image, (x1, y1), (x2, y2), (0, 255, 0), 2)

# Save the resulting image
cv2.imwrite('receipt_edges.png', image)


True

In [3]:
img.shape, scanner.shape, result.shape

((2599, 1731, 3), (2599, 1731), (2599, 1731))

In [2]:
import pandas as pd

df = pd.read_csv("/Users/liujiaen/Documents/Text_Recognition/dataset/findit/FindIt-Dataset-Train/T1-train/train.csv")
df.head()
print(df.shape)
len(df[df["correct"] == True])

(469, 7)


345

In [ ]:
correct = df[df["correct"] == True]
correct.to_csv("correct.csv", index=False)

In [2]:
# Test easy ocr

from easyocr import Reader
import cv2

# Read the image
image = cv2.imread("/home/jiaenliu/final_project/testimage/4.jpg")

# OCR the image
reader = Reader(['en'], gpu=True)
result = reader.readtext(image,)

# Print the result
print(result)


[([[422, 690], [1185, 690], [1185, 782], [422, 782]], 'CRF-CITY LA ROCHELLE', 0.7758780484401733), ([[421, 780], [1187, 780], [1187, 876], [421, 876]], '33 RUE DE LA SCIERIE', 0.6374939756836588), ([[427, 871], [627, 871], [627, 959], [427, 959]], '17000', 0.9733525592215241), ([[759, 873], [1185, 873], [1185, 965], [759, 965]], 'LA ROCHELLE', 0.9810264028810408), ([[424, 962], [550, 962], [550, 1050], [424, 1050]], 'Tel', 0.9839941354527475), ([[587, 990], [616, 990], [616, 1037], [587, 1037]], "'", 0.11441381604890921), ([[645, 962], [1185, 962], [1185, 1053], [645, 1053]], '05 , 46, 27 ,02, 12', 0.4825177609498993), ([[1277, 1032], [1564, 1032], [1564, 1141], [1277, 1141]], "'", 0.3146761157580933), ([[819, 1171], [1331, 1171], [1331, 1287], [819, 1287]], '', 0.0), ([[116, 1317], [447, 1317], [447, 1398], [116, 1398]], 'DESCRIPTIOA', 0.5370612612450623), ([[831, 1331], [1056, 1331], [1056, 1422], [831, 1422]], '.', 0.12504872222509622), ([[1092, 1323], [1195, 1323], [1195, 1396], [1

In [4]:
with open("output.txt", "w") as f:
    for item in result:
        text = str(item[1]).replace("\n", "")
        f.write(text + "\n")

In [19]:
test = reader.detect(image)
print(test)

([[[422, 1185, 690, 782], [421, 1187, 780, 876], [427, 627, 871, 959], [759, 1185, 873, 965], [424, 550, 962, 1050], [587, 616, 990, 1037], [645, 1185, 962, 1053], [1277, 1564, 1032, 1141], [819, 1331, 1171, 1287], [116, 447, 1317, 1398], [831, 1056, 1331, 1422], [1092, 1195, 1323, 1396], [1453, 1663, 1327, 1395], [87, 641, 1498, 1582], [1509, 1667, 1507, 1579], [88, 609, 1592, 1664], [1510, 1663, 1599, 1666], [177, 524, 1773, 1851], [677, 832, 1778, 1850], [846, 1058, 1778, 1850], [1485, 1641, 1782, 1849], [90, 816, 2049, 2140], [873, 1005, 2052, 2140], [1428, 1637, 2055, 2143], [313, 404, 2235, 2320], [427, 819, 2235, 2320], [842, 1500, 2232, 2323], [203, 898, 2321, 2418], [1059, 1414, 2324, 2415], [274, 1462, 2409, 2529], [164, 476, 2601, 2689], [1557, 1579, 2730, 2743], [274, 1044, 2773, 2881], [1354, 1562, 2781, 2870], [232, 356, 2963, 3031], [484, 579, 2963, 3031], [706, 889, 2959, 3032], [1012, 1310, 2959, 3034], [1404, 1642, 2959, 3032], [612, 1418, 3232, 3328], [846, 883, 3343

In [21]:
len(test[1])

1

In [23]:
print(test[0])

[[[422, 1185, 690, 782], [421, 1187, 780, 876], [427, 627, 871, 959], [759, 1185, 873, 965], [424, 550, 962, 1050], [587, 616, 990, 1037], [645, 1185, 962, 1053], [1277, 1564, 1032, 1141], [819, 1331, 1171, 1287], [116, 447, 1317, 1398], [831, 1056, 1331, 1422], [1092, 1195, 1323, 1396], [1453, 1663, 1327, 1395], [87, 641, 1498, 1582], [1509, 1667, 1507, 1579], [88, 609, 1592, 1664], [1510, 1663, 1599, 1666], [177, 524, 1773, 1851], [677, 832, 1778, 1850], [846, 1058, 1778, 1850], [1485, 1641, 1782, 1849], [90, 816, 2049, 2140], [873, 1005, 2052, 2140], [1428, 1637, 2055, 2143], [313, 404, 2235, 2320], [427, 819, 2235, 2320], [842, 1500, 2232, 2323], [203, 898, 2321, 2418], [1059, 1414, 2324, 2415], [274, 1462, 2409, 2529], [164, 476, 2601, 2689], [1557, 1579, 2730, 2743], [274, 1044, 2773, 2881], [1354, 1562, 2781, 2870], [232, 356, 2963, 3031], [484, 579, 2963, 3031], [706, 889, 2959, 3032], [1012, 1310, 2959, 3034], [1404, 1642, 2959, 3032], [612, 1418, 3232, 3328], [846, 883, 3343,